<a href="https://colab.research.google.com/github/Adwlluiz/Analise_Ecomerce/blob/main/Tratamento_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-bigquery

In [ ]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd



In [ ]:
auth.authenticate_user()

In [ ]:
cliente = bigquery.Client(project='t1engenhariadados')

In [ ]:
query = """
SELECT 'customers' AS tabela,
  (
    COUNTIF(customer_id IS NULL) +
    COUNTIF(customer_city IS NULL) +
    COUNTIF(customer_state IS NULL)
  ) AS total_nulls
FROM t1engenhariadados.turma2_grupo8.customers

UNION ALL

SELECT 'orders' AS tabela,
  (
    COUNTIF(order_id IS NULL) +
    COUNTIF(customer_id IS NULL) +
    COUNTIF(order_status IS NULL) +
    COUNTIF(order_purchase_timestamp IS NULL) +
    COUNTIF(order_approved_at IS NULL) +
    COUNTIF(order_delivered_carrier_date IS NULL) +
    COUNTIF(order_delivered_customer_date IS NULL) +
    COUNTIF(order_estimated_delivery_date IS NULL)
  ) AS total_nulls
FROM t1engenhariadados.turma2_grupo8.orders

UNION ALL

SELECT 'order_items' AS tabela,
  (
    COUNTIF(order_item_id IS NULL) +
    COUNTIF(product_id IS NULL) +
    COUNTIF(seller_id IS NULL) +
    COUNTIF(shipping_limit_date IS NULL) +
    COUNTIF(price IS NULL) +
    COUNTIF(freight_value IS NULL)
  ) AS total_nulls
FROM t1engenhariadados.turma2_grupo8.order_items

UNION ALL

SELECT 'order_payments' AS tabela,
  (
    COUNTIF(payment_sequential IS NULL) +
    COUNTIF(payment_type IS NULL) +
    COUNTIF(payment_installments IS NULL) +
    COUNTIF(payment_value IS NULL)
  ) AS total_nulls
FROM t1engenhariadados.turma2_grupo8.order_payments

UNION ALL

SELECT 'order_reviews' AS tabela,
  (
    COUNTIF(review_id IS NULL) +
    COUNTIF(order_id IS NULL) +
    COUNTIF(review_score IS NULL) +
    COUNTIF(review_comment_title IS NULL) +
    COUNTIF(review_comment_message IS NULL) +
    COUNTIF(review_creation_date IS NULL) +
    COUNTIF(review_answer_timestamp IS NULL)
  ) AS total_nulls
FROM t1engenhariadados.turma2_grupo8.order_reviews

UNION ALL

SELECT 'products' AS tabela,
  (
    COUNTIF(product_id IS NULL) +
    COUNTIF(product_category_name IS NULL) +
    COUNTIF(product_name_lenght IS NULL) +
    COUNTIF(product_description_lenght IS NULL) +
    COUNTIF(product_photos_qty IS NULL) +
    COUNTIF(product_weight_g IS NULL) +
    COUNTIF(product_length_cm IS NULL) +
    COUNTIF(product_height_cm IS NULL) +
    COUNTIF(product_width_cm IS NULL)
  ) AS total_nulls
FROM t1engenhariadados.turma2_grupo8.products

UNION ALL

SELECT 'sellers' AS tabela,
  (
    COUNTIF(seller_id IS NULL) +
    COUNTIF(seller_zip_code_prefix IS NULL) +
    COUNTIF(seller_city IS NULL) +
    COUNTIF(seller_state IS NULL)
  ) AS total_nulls
FROM t1engenhariadados.turma2_grupo8.sellers;
"""


In [ ]:
df = cliente.query(query).to_dataframe()

In [ ]:
df.head()

,tabela,total_nulls
0,sellers,0
1,order_payments,0
2,order_items,0
3,products,2448
4,order_reviews,145903


In [ ]:
cliente2 = bigquery.Client(project='t1engenhariadados')

In [ ]:

oi = """
SELECT
  order_id,
  customer_id,
  order_status,
  order_purchase_timestamp,
  IFNULL(CAST(order_approved_at AS STRING), "NA") AS order_approved_at,
  IFNULL(CAST(order_delivered_carrier_date AS STRING), "NA") AS order_delivered_carrier_date,
  IFNULL(CAST(order_delivered_customer_date AS STRING), "NA") AS order_delivered_customer_date,
  IFNULL(CAST(order_estimated_delivery_date AS STRING), "NA") AS order_estimated_delivery_date
FROM t1engenhariadados.turma2_grupo8.orders;
"""

In [ ]:
dt = cliente2.query(oi).to_dataframe()

In [ ]:
dt.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,a2e4c44360b4a57bdff22f3a4630c173,8886130db0ea6e9e70ba0b03d7c0d286,approved,2017-02-06 20:18:17+00:00,2017-02-06 20:30:19+00,NA,NA,2017-03-01 00:00:00+00
1,132f1e724165a07f6362532bfb97486e,b2191912d8ad6eac2e4dc3b6e1459515,approved,2017-04-25 01:25:34+00:00,2017-04-30 20:32:41+00,NA,NA,2017-05-22 00:00:00+00
2,809a282bbd5dbcabb6f2f724fca862ec,622e13439d6b5a0b486c435618b2679e,canceled,2016-09-13 15:24:19+00:00,2016-10-07 13:16:46+00,NA,NA,2016-09-30 00:00:00+00
3,e5215415bb6f76fe3b7cb68103a0d1c0,b6f6cbfc126f1ae6723fe2f9b3751208,canceled,2016-10-22 08:25:27+00:00,NA,NA,NA,2016-10-24 00:00:00+00
4,71303d7e93b399f5bcd537d124c0bcfa,b106b360fe2ef8849fbbd056f777b4d5,canceled,2016-10-02 22:07:52+00:00,2016-10-06 15:50:56+00,NA,NA,2016-10-25 00:00:00+00


In [ ]:
DECLARE sql STRING;
DECLARE table_sqls ARRAY<STRING>;

-- 1) Gera os SELECTs de cada tabela/coluna
SET table_sqls = (
  SELECT ARRAY_AGG(
    FORMAT("""
      SELECT
        '%s' AS tabela,
        '%s' AS coluna,
        COUNTIF(%s IS NULL) AS qtd_nulos
      FROM `turma2_grupo8.%s`
    """, table_name, column_name, column_name, table_name)
  )
  FROM `turma2_grupo8`.INFORMATION_SCHEMA.COLUMNS
);

-- 2) Junta todos os SELECTs com UNION ALL
SET sql = ARRAY_TO_STRING(table_sqls, " UNION ALL ");

-- 3) Executa o SQL dinâmico
EXECUTE IMMEDIATE sql;
